In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from transformers import RobertaTokenizer, RobertaModel, ViTFeatureExtractor, ViTModel
from PIL import Image
import joblib
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# 定义设备 # Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 读取CSV文件    # Read CSV file
df = pd.read_csv("最终的记录.csv", encoding='utf-8', encoding_errors='ignore')

# 重命名列（确保与之前一致） # Rename columns (ensure consistency with previous ones)
df.columns = [
    '商品名称', '一级种类', '二级种类', '图片地址', 'sku', '文本描述', '折扣率',
    '折扣价', '价格', '星级', '销量', '收入', '评论', 'cc-1', 'cc-2',
    'cc-3', 'DRC-1', 'DRC-2', 'DRC-3', 'RCV-1', 'RCV-2',
    'RCV-3', 'RSV-1', 'RSV-2', 'RSV-3'
]

# 转换数据类型   # Data type conversion
numeric_columns = ['星级', '折扣率', '折扣价', '价格', '销量', '收入']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 重新计算收入（如果需要）  # Recalculate income (if necessary)
df['收入'] = df['折扣价'] * df['销量']
df['收入'] = np.log10(df['收入'].replace(0, 1))  # 避免log10(0)错误

# 数值特征   # Numerical Features
numerical_features = ['星级', '折扣价', '折扣率', '价格']

# 标准化数值特征   # Standardized numerical features
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# 保存Scaler以备后续使用  # Save the Scaler for future use
joblib.dump(scaler, 'scaler.pkl')

# 设置设备
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 初始化 BERT 模型和分词器 # Initialize BERT model and tokenizer
bert_model_name = r"/root/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594"
tokenizer_bert = AutoTokenizer.from_pretrained(bert_model_name)
model_bert = AutoModel.from_pretrained(bert_model_name, from_tf=False)  # 根据实际情况设置 from_tf
model_bert.to(device)
model_bert.eval()  # 设置为评估模式 # Set to evaluation mode

def extract_text_features(texts, model, tokenizer, device, batch_size=32, max_length=128):
    all_features = []
    num_samples = len(texts)
    num_batches = int(np.ceil(num_samples / batch_size))
    
    with torch.no_grad():  # 禁用梯度计算 # Disable gradient computation
        for batch_idx in tqdm(range(num_batches), desc="提取 BERT 文本特征"):
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, num_samples)
            batch_texts = texts[start_idx:end_idx]
            
            # Tokenize with fixed max_length and padding to max_length
            inputs = tokenizer(
                batch_texts,
                return_tensors='pt',
                padding='max_length',  # 固定填充到 max_length
                truncation=True,
                max_length=max_length,
            )
            inputs = {key: val.to(device) for key, val in inputs.items()}
            
            # 获取模型输出  # Obtain model output
            outputs = model(**inputs)
             
            # 对最后一个隐藏状态进行平均池化，得到每个样本的固定维度特征   # Perform average pooling on the last hidden state to obtain fixed-dimensional features for each sample
            encoded_text = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # [batch_size, hidden_size]
            
            # 添加到特征列表     # Add to feature list
            all_features.append(encoded_text)
     
    # 将所有批次的特征堆叠起来，得到 [num_samples, hidden_size]   # Stack the features of all batches to obtain [num_samples, hidden_size]
    all_features = np.vstack(all_features)
    return all_features


# 确保所有文本字段为字符串类型 # Ensure that all text fields are of string type
text_columns = ['商品名称', '文本描述', '一级种类', '二级种类', 'cc-1']
for col in text_columns:
    df[col] = df[col].astype(str)
    
# 构建“其他文本”特征   # Constructing "other text" features
other_texts = (
    df['商品名称'] + " " +
    df['一级种类'] + " " +
    df['二级种类'] + " " +
    df['RCV-1']
).tolist()

# 构建“文本描述”特征 # Constructing "text description" features
text_descs = df['文本描述'].tolist()

# 提取 BERT 的“其他文本”特征 # Extracting the "Other Text" features of BERT
bert_other_text_features = extract_text_features(
    other_texts,
    model_bert,
    tokenizer_bert,
    device,
    batch_size=32,
    max_length=128
)
df['other_text_features'] = list(bert_other_text_features)

# 提取 BERT 的“文本描述”特征  # Extracting the "text description" feature of BERT
bert_text_desc_features = extract_text_features(
    text_descs,
    model_bert,
    tokenizer_bert,
    device,
    batch_size=32,
    max_length=128
)
df['text_desc_features'] = list(bert_text_desc_features)

print("BERT 文本特征提取完成。")

import torchvision.models as models
import torchvision
from torchvision import transforms

# 图像特征提取 # Image feature extraction
def vit_encoding(image_dir, num_images=2194, device='cpu'):
    vit_list = []
    model_name = "/root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3"
    feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
    model = ViTModel.from_pretrained(model_name).to(device).eval()

    def process_image(image_path):
        try:
            image = Image.open(image_path).convert('RGB')
            return feature_extractor(images=image, return_tensors="pt")
        except Exception as e:
            print(f"无法打开图片 {image_path}: {e}")
            # 返回全零张量以保持尺寸一致   # Return a tensor of all zeros to maintain consistent dimensions
            return feature_extractor(images=Image.new('RGB', (224, 224), (0, 0, 0)), return_tensors="pt")

    for i in tqdm(range(num_images), desc="提取图像特征"):
        image_path = os.path.join(image_dir, f"{i}.jpg")
        inputs = process_image(image_path)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            cls_token = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # [1, hidden_size]
            vit_list.append(cls_token.squeeze())
    
    vit_np = np.array(vit_list)  # [num_images, hidden_size]
    
    # PCA降维  # Return a tensor of all zeros to maintain consistent dimensions
    vit_pca = PCA(n_components=50)
    vit_features_pca = vit_pca.fit_transform(vit_np)
    
    # 保存PCA模型 # Save PCA model
    joblib.dump(vit_pca, 'vit_pca.pkl')
    
    return vit_features_pca

# 使用您的图像路径定义  # Define using your image path
image_directory = r"/root/img"
vit_features = vit_encoding(image_directory, num_images=len(df))
df['vit_features'] = list(vit_features)

import torchvision.models as models
import torchvision
from torchvision import transforms
# VGG特征提取函数  # VGG feature extraction function
def vgg_encoding(image_dir, num_images=2194, device='cpu', pca_components=50):
    """
    使用预训练的VGG16模型提取图像特征，并进行PCA降维。

    参数:
        image_dir (str): 存放图像的目录路径。
        num_images (int): 要处理的图像数量。
        device (str): 计算设备（'cpu'或'cuda'）。
        pca_components (int): PCA降维后的维度数。

    返回:
        np.ndarray: 降维后的VGG特征。
    """
     """
    Extract image features using a pre-trained VGG16 model and perform PCA dimensionality reduction.

    Parameters:
        image_dir (str): Directory path where images are stored.
        num_images (int): The number of images to be processed.
        device (str): Computing device ('cpu' or 'cuda').
        pca_components (int): The number of dimensions after PCA dimensionality reduction.

    Return:
        np.ndarray: VGG features after dimensionality reduction.
    """
    vgg_list = []

    # 初始化预训练的VGG16模型   # Initialize the pre-trained VGG16 model
    model = models.vgg16(pretrained=False)
    model.load_state_dict(torch.load(r"/root/.cache/torch/hub/checkpoints/vgg16-397923af.pth"))
    model.to(device).eval()

    # 定义预处理步骤  # Define preprocessing steps
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    def extract_features(image_path):
        """
        从图像路径提取VGG特征。

        参数:
            image_path (str): 图像文件路径。

        返回:
            np.ndarray: 提取的特征向量。
        """
         """
        Extract VGG features from the image path.

        Parameters:
            image_path (str): Path to the image file.

        Return:
            np.ndarray: Extracted feature vector.
        """
        
        try:
            image = Image.open(image_path).convert('RGB')
            tensor = preprocess(image)
        except Exception as e:
            print(f"无法打开图片 {image_path}: {e}")
            # 返回全零张量以保持尺寸一致
            tensor = preprocess(Image.new('RGB', (224, 224), (0, 0, 0)))
        
        tensor = tensor.unsqueeze(0).to(device)  # 添加批次维度并移动到设备 # Add batch dimension and move to device
        with torch.no_grad():
            features = model(tensor)
        embedding = features.view(features.size(0), -1).cpu().numpy()  # 展平并转换为numpy数组  # Flatten and convert to numpy array
        return embedding.squeeze()  # 移除多余的维度  # Traverse all images and extract features

    # 遍历所有图像并提取特征
    for i in tqdm(range(num_images), desc="提取VGG图像特征"):
        image_path = os.path.join(image_dir, f"{i}.jpg")
        features = extract_features(image_path)
        vgg_list.append(features)
    
    vgg_np = np.array(vgg_list)  # [num_images, feature_dim]

    # PCA降维
    vgg_pca = PCA(n_components=pca_components)
    vgg_features_pca = vgg_pca.fit_transform(vgg_np)
 
    # 保存PCA模型
    joblib.dump(vgg_pca, 'vgg_pca.pkl')

    return vgg_features_pca

# 提取vgg特征  # Extracting VGG features
vgg_features = vgg_encoding(image_directory, num_images=len(df), device='cuda' if torch.cuda.is_available() else 'cpu')
df['vgg_features'] = list(vgg_features)

def shuff_encoding(image_dir, num_images=2194, device='cpu', pca_components=50):

    shuff_list = []

    # 定义图像预处理    # Define image preprocessing
    transform = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # 加载预训练的ShuffleNet模型 # Load the pre-trained ShuffleNet model
    model = models.shufflenet_v2_x1_0(pretrained=False)
    model.load_state_dict(torch.load(r"/root/.cache/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth"))
    model.to(device).eval()

    def load_image(image_path):
        try:
            image = Image.open(image_path).convert('RGB')
            return transform(image).unsqueeze(0).to(device)  # 添加批次维度并移动到设备  # Add batch dimension and move to device
        except Exception as e:
            print(f"无法加载图片 {image_path}: {e}")
            return transform(Image.new('RGB', (299, 299), (0, 0, 0))).unsqueeze(0).to(device)

    # 遍历所有图像并提取特征     # Traverse all images and extract features
    for i in tqdm(range(num_images), desc="提取ShuffleNet图像特征"):
        image_path = os.path.join(image_dir, f"{i}.jpg")
        image = load_image(image_path)
        with torch.no_grad():
            features = model(image)
        features = features.view(-1).cpu().numpy()  # 展平并转换为numpy数组
        shuff_list.append(features)

    shuff_np = np.array(shuff_list)

    # PCA降维
    shuff_pca = PCA(n_components=pca_components)
    shuff_features_pca = shuff_pca.fit_transform(shuff_np)

    # 保存PCA模型
    joblib.dump(shuff_pca, 'shuff_pca.pkl')

    return shuff_features_pca

# 提取shuff特征   # Extract shuff features
shuff_features = shuff_encoding(image_directory, num_images=len(df), device='cuda' if torch.cuda.is_available() else 'cpu')
df['shuff_features'] = list(shuff_features)

def resnet_encoding(image_dir, num_images=2194, device='cpu', pca_components=50):

    resnet_list = []

    # 定义图像预处理
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # 加载预训练的ResNet50模型    # Load pre-trained ResNet50 model
    model = models.resnet50(pretrained=False)
    model.load_state_dict(torch.load(r"/root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth"))
    model.to(device).eval()

    def load_image(image_path):
        try:
            image = Image.open(image_path).convert('RGB')
            return transform(image).unsqueeze(0).to(device)  # 添加批次维度并移动到设备
        except Exception as e:
            print(f"无法加载图片 {image_path}: {e}")
            return transform(Image.new('RGB', (224, 224), (0, 0, 0))).unsqueeze(0).to(device)

    # 遍历所有图像并提取特征    # Traverse all images and extract features
    for i in tqdm(range(num_images), desc="提取ResNet50图像特征"):
        image_path = os.path.join(image_dir, f"{i}.jpg")
        image = load_image(image_path)
        with torch.no_grad():
            features = model(image)
        features = features.view(-1).cpu().numpy()  # 展平并转换为numpy数组
        resnet_list.append(features)

    resnet_np = np.array(resnet_list)

    # PCA降维
    resnet_pca = PCA(n_components=pca_components)
    resnet_features_pca = resnet_pca.fit_transform(resnet_np)
    
    # 保存PCA模型
    joblib.dump(resnet_pca, 'resnet_pca.pkl')

    return resnet_features_pca


# 提取resnet特征    # Extracting ResNet features
resnet_features = resnet_encoding(image_directory, num_images=len(df), device='cuda' if torch.cuda.is_available() else 'cpu')
df['resnet_features'] = list(resnet_features)

# 定义Dataset类
class IncomeDataset(Dataset):
    def __init__(self, df, image_feature_type='vit'):
        """
        初始化数据集。
        参数:
            df (pd.DataFrame): 数据框。
            image_feature_type (str): 要使用的图像特征类型，支持 'vit'、'vgg'、'shuff'、'resnet'。
        """
        """
        Initialize the dataset.
        Parameters:
            df (pd.DataFrame): Data frame.
            image_feature_type (str): The type of image feature to be used, supported by 'vit', 'vgg', 'shuff', and 'resnet'.
        """
        
        supported_features = ['vit_features', 'vgg_features', 'shuff_features', 'resnet_features']
        feature_mapping = {
            'vit': 'vit_features',
            'vgg': 'vgg_features',
            'shuff': 'shuff_features',
            'resnet': 'resnet_features'
        }
        if image_feature_type not in feature_mapping:
            raise ValueError(f"Unsupported image feature type: {image_feature_type}. Supported types: {list(feature_mapping.keys())}")
        selected_feature = feature_mapping[image_feature_type]
        self.image_features = np.stack(df[selected_feature].values)
        self.other_text_features = np.stack(df['other_text_features'].values)
        self.text_desc_features = np.stack(df['text_desc_features'].values)
        self.numerical_features = df[[ '折扣价', '折扣率', '价格']].values
        self.targets = df['收入'].values.astype(np.float32)
        self.image_feature_type = image_feature_type

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        image = self.image_features[idx]
        other_text = self.other_text_features[idx]
        text_desc = self.text_desc_features[idx]
        numerical = self.numerical_features[idx]
        target = self.targets[idx]
        return {
            'image': torch.tensor(image, dtype=torch.float32),
            'other_text': torch.tensor(other_text, dtype=torch.float32),
            'text_desc': torch.tensor(text_desc, dtype=torch.float32),
            'numerical': torch.tensor(numerical, dtype=torch.float32),
            'target': torch.tensor(target, dtype=torch.float32)
        }

# PSI framework
class RegressionModelWithAttention(nn.Module):
    def __init__(self, input_dim_image, input_dim_other_text, input_dim_text_desc, input_dim_numerical, attention_dim=128):
        super(RegressionModelWithAttention, self).__init__()
        # 图像特征处理  # Image feature processing
        self.image_dense = nn.Sequential(
            nn.Linear(input_dim_image, 256),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        # 其他文本特征处理 # Other text feature processing
        self.other_text_dense = nn.Sequential(
            nn.Linear(input_dim_other_text, 256),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        # “文本描述”特征处理（移除自注意力层）  # "Text description" feature processing (removing the self-attention layer)
        self.text_dense = nn.Sequential(
            nn.Linear(input_dim_text_desc, 256),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        # 数值特征处理 # Numerical feature processing
        self.numerical_dense = nn.Sequential(
            nn.Linear(input_dim_numerical, 64),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        # 融合特征    # Fusion featuresArithmeticError
        self.fusion = nn.Sequential(
            nn.Linear(256 + 256 , 256),  # 图像特征 + 文本描述特征 # Image features + textual descriptive features
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.combined = nn.Sequential(
            nn.Linear(256 + 256 + 64, 128),  # 其他文本特征 + 融合特征 + 数值特征 # Other text features + fused features + numerical features
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        # 输出层 # Output layer
        self.output = nn.Linear(128, 1)  # 回归输出  # Regression output

    def forward(self, image, other_text, text_desc, numerical):
        # 图像特征处理   # Image feature processing
        image_features = self.image_dense(image)  # [batch_size, 256]
        # 其他文本特征处理   # Other text feature processing
        other_text_features = self.other_text_dense(other_text)  # [batch_size, 256]
        # “文本描述”特征处理（直接传递给全连接层） # "Text description" feature processing (directly passed to the fully connected layer)
        text_desc_features = self.text_dense(text_desc)  # [batch_size, 256]
        # 数值特征处理  # Numerical feature processing
        numerical_features = self.numerical_dense(numerical)  # [batch_size, 64]
        # 融合图像、其他文本和文本描述特征 # Integrate image, other text, and textual descriptive features
        fused = torch.cat((image_features, text_desc_features), dim=1)  # [batch_size, 768]
        fused = self.fusion(fused)  # [batch_size, 256]
        # 与数值特征拼接  # Concatenation with numerical features
        combined = torch.cat((fused, numerical_features,other_text_features), dim=1)  # [batch_size, 320]
        combined = self.combined(combined)  # [batch_size, 128]
        # 输出  # output
        output = self.output(combined)  # [batch_size, 1]
        return output

# 定义评估函数 # Define evaluation function
def evaluate_model(model, dataloader, device):
    model.eval()  # 设置模型为评估模式 # Set the model to evaluation mode
    all_predictions = []
    all_targets = []
    with torch.no_grad():  # 关闭梯度计算 # Disable gradient computation
        for batch in dataloader:
            image = batch['image'].to(device)
            other_text = batch['other_text'].to(device)
            text_desc = batch['text_desc'].to(device)
            numerical = batch['numerical'].to(device)
            targets = batch['target'].to(device)
            outputs = model(image, other_text, text_desc, numerical)
            all_predictions.extend(outputs.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    # 转换为NumPy数组并应用反变换（假设使用了对数变换） # Convert to NumPy array and apply inverse transformation (assuming logarithmic transformation is used)
    all_predictions = 10**np.array(all_predictions).flatten()
    all_targets = 10**np.array(all_targets).flatten()
    # 计算指标   # Calculation Metrics
    mse = mean_squared_error(all_targets, all_predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(all_targets, all_predictions)
    r2 = r2_score(all_targets, all_predictions)
    return rmse, mae, r2, all_targets, all_predictions


# 定义训练和评估函数   # Define training and evaluation functions
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=100):
    """
    训练和评估模型。

    参数:
        model (nn.Module): 要训练的模型。
        train_loader (DataLoader): 训练数据加载器。
        val_loader (DataLoader): 验证数据加载器。
        criterion: 损失函数。
        optimizer: 优化器。
        device (str): 设备类型。
        num_epochs (int): 训练轮数。

    返回:
        train_losses, val_losses, rmse, mae, r2
    """
    """
    Train and evaluate the model.

    Parameters:
        model (nn.Module): The model to be trained.
        train_loader (DataLoader): Training data loader.
        val_loader (DataLoader): Validation data loader.
        criterion: loss function.
        optimizer: Optimizer.
        device (str): Device type.
        num_epochs (int): Number of training epochs.

    Return:
        train_losses, val_losses, rmse, mae, r2
    """
    
    train_losses = []
    val_losses = []
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            optimizer.zero_grad()
            outputs = model(
                batch['image'].to(device),
                batch['other_text'].to(device),
                batch['text_desc'].to(device),
                batch['numerical'].to(device)
            ).squeeze()
            loss = criterion(outputs, batch['target'].to(device))
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * batch['image'].size(0)
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        # 验证阶段
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                outputs = model(
                    batch['image'].to(device),
                    batch['other_text'].to(device),
                    batch['text_desc'].to(device),
                    batch['numerical'].to(device)
                ).squeeze()
                loss = criterion(outputs, batch['target'].to(device))
                val_loss += loss.item() * batch['image'].size(0)
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

    # 在所有epoch结束后进行评估  # Evaluate after all epochs have ended
    rmse, mae, r2, _, _ = evaluate_model(model, val_loader, device)
    return train_losses, val_losses, rmse, mae, r2

# 设置随机种子 # Set random seed
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from transformers import set_seed
SEEDS = [ 42, 23, 15, 34, 18, 32, 47, 27,8, 52]


# 定义不同图像特征类型  # Define different types of image features
image_feature_types = ['vit', 'vgg', 'shuff', 'resnet']  # 假设 'resnet_features' 已提取

# 存储实验结果 # Store experimental results
experiment_results = {
    'feature_type': [],
    'seed': [],
    'final_train_loss': [],
    'final_val_loss': [],
    'RMSE': [],
    'MAE': [],
    'R2': []
}

# 定义模型参数（根据不同特征的维度进行调整）# Define model parameters (adjusted according to the dimensions of different features)
FEATURE_DIMENSIONS = {
    'vit': 50,          # PCA后的ViT特征维度 # ViT feature dimension after PCA
    'vgg': 50,          # PCA后的VGG特征维度  # VGGfeature dimension after PCA
    'shuff': 50,        # PCA后的ShuffleNet特征维度  # ShuffleNet feature dimension after PCA
    'resnet': 50        # PCA后的ResNet特征维度   # ResNet feature dimension after PCA
}
input_dim_other_text = 768  # BERT的hidden_size # The hidden_size of BERT
input_dim_text_desc = 768   # 根据实际hidden_size调整  Adjust according to the actual hidden_size
input_dim_numerical = 3     # 数值特征数量  Number of numerical features

# 定义训练参数  # Define training parameters
batch_size = 32
num_epochs = 1000

# 定义损失函数   # Define loss function
criterion = nn.MSELoss()

# 检查设备  # Check equipment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 遍历所有随机种子    # Iterate over all random seeds
for seed in SEEDS:
    print(f"\n=== 使用随机种子 {seed} 进行数据划分 ===")
    set_seed(seed)

    # 数据集划分  # Dataset partitioning
    train_df, val_df = train_test_split(
        df,
        test_size=0.01,  # 根据需要调整验证集比例    # Adjust the validation set ratio as needed
        random_state=seed,
        shuffle=True
    )

    # 遍历所有图像特征类型   # Iterate over all image feature types
    for feature_type in image_feature_types:
        print(f"\n=== 训练使用 {feature_type} 特征的模型 ===")

        # 定义输入维度   # Define input dimensions
        input_dim_image = FEATURE_DIMENSIONS.get(feature_type)
        if input_dim_image is None:
            print(f"未定义特征类型 {feature_type} 的输入维度。跳过...")
            continue

        # 创建训练集和验证集  # Create training set and validation set
        train_dataset = IncomeDataset(train_df, image_feature_type=feature_type)
        val_dataset = IncomeDataset(val_df, image_feature_type=feature_type)

        # 创建数据加载器    # Create a data loader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

        # 实例化模型      # Instantiate Model
        model = RegressionModelWithAttention(
            input_dim_image=input_dim_image,
            input_dim_other_text=input_dim_other_text,
            input_dim_text_desc=input_dim_text_desc,
            input_dim_numerical=input_dim_numerical,
            attention_dim=128
        ).to(device)

        # 定义优化器   # Train and evaluate the model
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

        # 训练和评估模型
        train_losses, val_losses, rmse, mae, r2 = train_and_evaluate(
            model,
            train_loader,
            val_loader,
            criterion,
            optimizer,
            device,
            num_epochs=num_epochs
        )

        # 记录最终的训练和验证损失以及评估指标  # Record the final training and validation losses as well as evaluation metrics
        experiment_results['feature_type'].append(feature_type)
        experiment_results['seed'].append(seed)
        experiment_results['final_train_loss'].append(train_losses[-1])
        experiment_results['final_val_loss'].append(val_losses[-1])
        experiment_results['RMSE'].append(rmse)
        experiment_results['MAE'].append(mae)
        experiment_results['R2'].append(r2)

        # 保存模型（可选） # Save the model (optional)
        model_save_path = f"regression_model_{feature_type}_seed{seed}.pth"
        torch.save(model.state_dict(), model_save_path)
        print(f"模型已保存为 {model_save_path}")
        print(f"评估指标 - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

# 将实验结果转换为DataFrame
results_df = pd.DataFrame(experiment_results)
# 可选：保存实验结果到CSV文件  # Optional: Save the experimental results to a CSV file
results_df.to_csv("RCV-1_experiment_results.csv", index=False, encoding='utf-8-sig')

提取 BERT 文本特征: 100%|██████████| 69/69 [00:05<00:00, 13.30it/s]
/root/miniconda3/lib/python3.12/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


BERT 文本特征提取完成。


Some weights of ViTModel were not initialized from the model checkpoint at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
提取图像特征:  29%|██▉       | 641/2194 [02:42<08:10,  3.16it/s]/root/miniconda3/lib/python3.12/site-packages/PIL/Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
提取图像特征: 100%|██████████| 2194/2194 [10:11<00:00,  3.59it/s]
/root/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning

Using device: cuda

=== 使用随机种子 42 进行数据划分 ===

=== 训练使用 vit 特征的模型 ===


Epoch 1/1000 - Validation: 100%|██████████| 1/1 [00:00<00:00, 319.52it/s]


...

Epoch 81/1000 - Validation: 100%|██████████| 1/1 [00:00<00:00, 234.92it/s]


Epoch 81/1000 - Train Loss: 0.1040 - Val Loss: 0.2624


Epoch 82/1000 - Validation: 100%|██████████| 1/1 [00:00<00:00, 242.82it/s]


Epoch 82/1000 - Train Loss: 0.1058 - Val Loss: 0.1741


Epoch 83/1000 - Training:  57%|█████▋    | 39/68 [00:00<00:00, 126.56it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

